In [ ]:
! pip install -q datasets ragas langchain_openai

In [ ]:
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key: ')

In [ ]:
from datasets import load_dataset
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate, EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
dataset = load_dataset("explodinggradients/amnesty_qa", "english_v3", trust_remote_code=True)

In [ ]:
eval_dataset = EvaluationDataset.from_hf_dataset(dataset["eval"])

In [ ]:
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [ ]:
metrics = [
    LLMContextRecall(llm=evaluator_llm), 
    FactualCorrectness(llm=evaluator_llm), 
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings)
]
results = evaluate(dataset=eval_dataset, metrics=metrics)

In [ ]:
df = results.to_pandas()
df.head()

In [ ]:
df